In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
bnf_pandas = pd.read_csv("../data/indices/NIFTY 50.csv")

In [3]:
# symbol = 'midcp'
symbol = 'nifty'
# symbol = 'fnf'
# symbol = 'bnf'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 8
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1000000
print(LEVERAGE_)

7


In [4]:
bnf_pandas.head()

,datetime,o,h,l,c
0,2017-01-02T09:15:00.000000,8210.1000,8211.7000,8189.0000,8189.5500
1,2017-01-02T09:16:00.000000,8188.7500,8193.9500,8188.7500,8189.9500
2,2017-01-02T09:17:00.000000,8190.1500,8190.7500,8173.7000,8173.7000
3,2017-01-02T09:18:00.000000,8173.3500,8177.5500,8169.1500,8177.5500
4,2017-01-02T09:19:00.000000,8177.8500,8178.1500,8173.4500,8174.4000


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

In [9]:
bnf2 = resample(bnf, '15m')
bnf_pandas = bnf2.to_pandas()
bnf_pandas.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)

In [10]:
def ma_crossover_logic(df, fast_ma, slow_ma):
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    
    fast_ema = df['close'].rolling(fast_ma).mean()
    slow_ema = df['close'].rolling(slow_ma).mean()
    df['fast_ma'] = fast_ema
    df['slow_ma'] = slow_ema
    
    df['signal'] = 0  # Default to no signal
    df.loc[fast_ema > slow_ema, 'signal'] = 1   # Long Signal
    df.loc[fast_ema < slow_ema, 'signal'] = -1  # Short Signal
    
    return df

In [11]:
def execute(df, sl_pct, n, portfolio=100000, leverage=1, lot_size=1, slippage=0.0001, rpt = 1):
    
    trade_book = []
    in_trade_long = False
    in_trade_short = False
    signal_initial_sl_long = 0
    signal_initial_sl_short = 0
    cumulative_roi = 0
    max_drawdown = 0
    peak_roi = 0

    # print(df.tail(50).to_string())

    for i in range(1, len(df)):
        points = 0
        
        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]

        previous_candle_open = df.iloc[i-1]["open"]
        previous_candle_high = df.iloc[i-1]["high"]
        previous_candle_low = df.iloc[i-1]["low"]
        previous_candle_close = df.iloc[i-1]["close"]

        if not in_trade_long:
            if (df.iloc[i-1]['signal'] == 1) and current_candle_high >= previous_candle_high and current_candle_open <= previous_candle_high:
                # Entry Triggered
                entry_price_long = previous_candle_high
                initial_sl_long = entry_price_long * (1 - (sl_pct / 100))
                # initial_sl_long = df.iloc[max(0, i-n):i]['low'].min()
                signal_generation_time_long = df.iloc[i-1]['datetime']
                entry_time_long = df.iloc[i]['datetime']
                in_trade_long = True

        if in_trade_long:

            if current_candle_low <= initial_sl_long:
                if current_candle_open > initial_sl_long:
                    # Initial SL Hit
                    in_trade_long = False
                    exit_price_long = initial_sl_long
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
                elif current_candle_open < initial_sl_long  and (df.iloc[i]['datetime'] != entry_time_long):
                    # Gap Open SL
                    in_trade_long = False
                    exit_price_long = current_candle_open
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'Gap SL Hit'
                else:
                    # Initial SL Hit
                    in_trade_long = False
                    exit_price_long = initial_sl_long
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
                    
            elif df.iloc[i-1]['signal'] == -1 and current_candle_low <= previous_candle_low and previous_candle_low > initial_sl_long:
                # MA Cross in Opposite Direction
                if current_candle_open >= previous_candle_low:
                    in_trade_long = False
                    exit_price_long = previous_candle_low
                    exit_time_long = df.iloc[i]['datetime']
                    points = exit_price_long - entry_price_long
                    remarks = 'MA Cross Opp'
                else:
                    if current_candle_high >= previous_candle_low:
                        in_trade_long = False
                        exit_price_long = previous_candle_low
                        exit_time_long = df.iloc[i]['datetime']
                        points = exit_price_long - entry_price_long
                        remarks = 'MA Cross Opp'
                    else:
                        in_trade_long = False
                        exit_price_long = current_candle_close
                        exit_time_long = df.iloc[i]['datetime']
                        points = exit_price_long - entry_price_long
                        remarks = 'MA Cross Opp W Gap Exit'

            if not in_trade_long and points:
                # Exit Found
                # qty = int(round((portfolio * leverage / entry_price_long) / lot_size)) * lot_size
                qty = int(round((portfolio * rpt / 100) / abs(entry_price_long - initial_sl_long)) / lot_size) * lot_size
                slippage_ = slippage * (entry_price_long + exit_price_long)
                final_points = points - slippage_
                pnl = final_points * qty
                roi = (pnl / portfolio) * 100
                
                trade_book.append({
                    "Trade Type": "LONG",
                    "Entry Time": entry_time_long,
                    "Entry Price": entry_price_long,
                    "Initial SL": initial_sl_long,
                    "Exit Time": exit_time_long,
                    "Exit Price": exit_price_long,
                    "Points Captured": points,
                    "Slippage": slippage_,
                    "Qty": qty,
                    "Final Points": final_points,
                    "PnL": pnl,
                    "ROI%": roi,
                    "Trade Duration": exit_time_long - entry_time_long,
                    "Remarks": remarks,
                })

                remarks = ""
                points = 0

        if not in_trade_short:
            if (df.iloc[i-1]['signal'] == -1) and current_candle_low <= previous_candle_low:
                # Entry Triggered for Short
                entry_price_short = previous_candle_low
                initial_sl_short = entry_price_short * (1 + (sl_pct / 100))
                # initial_sl_short = df.iloc[max(0, i-n):i]['high'].max()
                entry_time_short = df.iloc[i]['datetime']
                in_trade_short = True

        if in_trade_short:
            
            if current_candle_high >= initial_sl_short:
                if current_candle_open < initial_sl_short:
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = initial_sl_short
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'ISL Hit'
                elif current_candle_open < initial_sl_short and (df.iloc[i]['datetime'] != entry_time_short):
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = current_candle_open
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'Gap SL Hit'
                else:
                    # Initial SL Hit for Short
                    in_trade_short = False
                    exit_price_short = initial_sl_short
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'ISL Hit'
                    
            elif df.iloc[i-1]['signal'] == 1 and current_candle_high >= previous_candle_high and previous_candle_high < initial_sl_short:
                # MA Cross in Opposite Direction for Short
                if current_candle_open <= previous_candle_high:
                    in_trade_short = False
                    exit_price_short = previous_candle_high
                    exit_time_short = df.iloc[i]['datetime']
                    points = entry_price_short - exit_price_short
                    remarks = 'MA Cross Opp'
                else:
                    if current_candle_low <= previous_candle_high:
                        in_trade_short = False
                        exit_price_short = previous_candle_high
                        exit_time_short = df.iloc[i]['datetime']
                        points = entry_price_short - exit_price_short
                        remarks = 'MA Cross Opp'
                    else:
                        in_trade_short = False
                        exit_price_short = current_candle_close
                        exit_time_short = df.iloc[i]['datetime']
                        points = entry_price_short - exit_price_short
                        remarks = 'MA Cross Opp W Gap Exit'

            if not in_trade_short and points:
                # Exit Found
                # qty = int(round((portfolio * leverage / entry_price_short) / lot_size)) * lot_size
                qty = int(round((portfolio * rpt / 100) / abs(entry_price_short - initial_sl_short)) / lot_size) * lot_size
                slippage_ = slippage * (entry_price_short + exit_price_short)
                final_points = points - slippage_
                pnl = final_points * qty
                roi = (pnl / portfolio) * 100

                trade_book.append({
                    "Trade Type": "SHORT",
                    "Entry Time": entry_time_short,
                    "Entry Price": entry_price_short,
                    "Initial SL": initial_sl_short,
                    "Exit Time": exit_time_short,
                    "Exit Price": exit_price_short,
                    "Points Captured": points,
                    "Slippage": slippage_,
                    "Qty": qty,
                    "Final Points": final_points,
                    "PnL": pnl,
                    "ROI%": roi,
                    "Trade Duration": exit_time_short - entry_time_short,
                    "Remarks": remarks,
                })

                remarks = ""
                points = 0

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [12]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [15]:
PORTFOLIO = 50_00_000
LEVERAGE_ = 7
LOT_SIZE_ = 75
SLIPPAGE_ = 0.0002
RPT_ = 2

In [25]:
stats_dictionary = {}

slow_ma = 20
fast_ma = 8
# trailing_ma = 80
sl_pct = 1

df = ma_crossover_logic(bnf_pandas, fast_ma, slow_ma)

tb = execute(df, sl_pct, 10, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, RPT_)
variation = f'MA1 : {slow_ma}, MA2 : {fast_ma}, SL : {sl_pct}%'

In [26]:
tb['Trade Year'] = tb['Entry Time'].dt.year
tb = tb.sort_values(by="Entry Time")
stats = generate_stats(tb, variation)

In [27]:
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                   Variation
2017     -26.7082          298  29.1946                1.0794             -0.5716     -30.5929      -0.8730  MA1 : 20, MA2 : 8, SL : 1%
2018      23.8701          281  38.0783                1.4493             -0.7541     -12.2964       1.9412  MA1 : 20, MA2 : 8, SL : 1%
2019      60.9276          267  40.4494                1.6261             -0.7213     -14.9592       4.0729  MA1 : 20, MA2 : 8, SL : 1%
2020      67.2462          293  39.5904                2.5549             -1.2945     -46.3978       1.4493  MA1 : 20, MA2 : 8, SL : 1%
2021      37.0723          277  40.4332                1.6689             -0.9081     -24.5934       1.5074  MA1 : 20, MA2 : 8, SL : 1%
2022     -38.6827          296  30.0676                1.8003             -0.9609     -45.7640      -0.8453  MA1 : 20, MA2 : 8, SL : 1%
2023       5.4268          281  36.2989         

In [28]:
# tb.tail(250)

In [29]:
tb['Trade Duration'].mean()

Timedelta('1 days 06:41:37.783572359')

In [21]:
# tb.to_csv('full_ma_cross_4_20_MAs_15m_TF.csv')

In [30]:
sl_pct_range = [0.5, 0.75, 1, 1.25, 1.5]
stats_dictionary = {}

PORTFOLIO = 1_00_00_000
LEVERAGE_ = 7
LOT_SIZE_ = 75
SLIPPAGE_ = 0.0002

for i in range(4, 41, 2):
    for j in range(6, 81, 2):
        for sl in sl_pct_range:
            if i < j and ((j-i) <= 16):
                variation = f'MA1 : {i}, MA2 : {j}, SL : {sl}%'
                print(variation)
                df = ma_crossover_logic(bnf_pandas, i, j)
                tb = execute(df, sl, 1, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, 2)
                if len(tb) > 0:
                    tb['Trade Year'] = tb['Entry Time'].dt.year
                    tb = tb.sort_values(by="Entry Time")
                    stats = generate_stats(tb, variation)

                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 15:
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 4, MA2 : 6, SL : 0.5%
MA1 : 4, MA2 : 6, SL : 0.75%
MA1 : 4, MA2 : 6, SL : 1%
MA1 : 4, MA2 : 6, SL : 1.25%
MA1 : 4, MA2 : 6, SL : 1.5%
MA1 : 4, MA2 : 8, SL : 0.5%
MA1 : 4, MA2 : 8, SL : 0.75%
MA1 : 4, MA2 : 8, SL : 1%
MA1 : 4, MA2 : 8, SL : 1.25%
MA1 : 4, MA2 : 8, SL : 1.5%
MA1 : 4, MA2 : 10, SL : 0.5%
MA1 : 4, MA2 : 10, SL : 0.75%
MA1 : 4, MA2 : 10, SL : 1%
MA1 : 4, MA2 : 10, SL : 1.25%
MA1 : 4, MA2 : 10, SL : 1.5%
MA1 : 4, MA2 : 12, SL : 0.5%
MA1 : 4, MA2 : 12, SL : 0.75%
MA1 : 4, MA2 : 12, SL : 1%
MA1 : 4, MA2 : 12, SL : 1.25%
MA1 : 4, MA2 : 12, SL : 1.5%
MA1 : 4, MA2 : 14, SL : 0.5%
MA1 : 4, MA2 : 14, SL : 0.75%
MA1 : 4, MA2 : 14, SL : 1%
MA1 : 4, MA2 : 14, SL : 1.25%
MA1 : 4, MA2 : 14, SL : 1.5%
MA1 : 4, MA2 : 16, SL : 0.5%
MA1 : 4, MA2 : 16, SL : 0.75%
MA1 : 4, MA2 : 16, SL : 1%
MA1 : 4, MA2 : 16, SL : 1.25%
MA1 : 4, MA2 : 16, SL : 1.5%
MA1 : 4, MA2 : 18, SL : 0.5%
MA1 : 4, MA2 : 18, SL : 0.75%
MA1 : 4, MA2 : 18, SL : 1%
MA1 : 4, MA2 : 18, SL : 1.25%
MA1 : 4, MA2 : 18, SL : 

KeyboardInterrupt: 